In [1]:
import pyspark
import os
import re

In [2]:
DATA_PATH = '../data/madrid_sim'

In [3]:
files = [f for f in os.listdir(DATA_PATH) if (os.path.isfile(os.path.join(DATA_PATH, f)) and re.match('\d_MNO_MNO1.csv', f[-14:]))]

In [4]:
sc = pyspark.SparkContext()

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
sqlContext = pyspark.SQLContext(sc)

In [7]:
sc.addFile(os.path.join(DATA_PATH, files[0]))

In [8]:
from pyspark.sql.types import *

schema = StructType([StructField("t", IntegerType(), True), 
                     StructField("AntennaId", IntegerType(), True), 
                     StructField("EventCode", IntegerType(), True),
                     StructField("PhoneId", IntegerType(), True), 
                     StructField("x", FloatType(), True),
                     StructField("y", FloatType(), True),
                     StructField("TileId", IntegerType(), True)
                   ])

In [13]:
main_df = sqlContext.read.csv(pyspark.SparkFiles.get(files[0]), header=True, schema=schema)

In [14]:
for i in range (1, len(files)):
    sc.addFile(os.path.join(DATA_PATH, files[i]))
    main_df = main_df.union(sqlContext.read.csv(pyspark.SparkFiles.get(files[i]), header=True, schema=schema))

In [11]:
main_df.show(100)

+---+---------+---------+-------+---------+---------+------+
|  t|AntennaId|EventCode|PhoneId|        x|        y|TileId|
+---+---------+---------+-------+---------+---------+------+
| 46|       73|        0|    242| 602098.7| 650285.4|   483|
| 46|       73|        0|    241| 602098.7| 650285.4|   483|
| 47|       73|        2|    242| 602098.7| 650285.4|   483|
| 47|       73|        2|    241| 602098.7| 650285.4|   483|
| 48|       73|        1|    242|601461.25| 649481.3|   260|
| 48|       73|        1|    241|601461.25| 649481.3|   260|
|  7|       95|        0|    268| 601408.6|650864.44|   638|
|  8|       95|        2|    268| 601408.6|650864.44|   638|
|  9|       95|        2|    268| 601408.6|650864.44|   638|
| 10|       95|        2|    268| 601408.6|650864.44|   638|
| 11|       95|        2|    268| 601408.6|650864.44|   638|
| 12|       95|        2|    268| 601408.6|650864.44|   638|
| 13|       95|        2|    268| 601408.6|650864.44|   638|
| 14|       95|        2

In [14]:
main_df.printSchema()

root
 |-- t: integer (nullable = true)
 |-- AntennaId: integer (nullable = true)
 |-- EventCode: integer (nullable = true)
 |-- PhoneId: integer (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- TileId: integer (nullable = true)



In [12]:
main_df.count()

1439

In [14]:
group_df = main_df.groupBy('t','AntennaId').count()

In [15]:
group_df.orderBy('PhoneId', 'AntennaId').show()

+---+---------+-----+
|  t|AntennaId|count|
+---+---------+-----+
|  0|        5|    1|
|  0|       10|    1|
|  0|       23|    2|
|  0|       39|    1|
|  0|       63|    1|
|  0|       77|    1|
|  0|       79|    2|
|  0|      101|    1|
|  0|      131|    1|
|  0|      133|    2|
|  0|      142|    1|
|  0|      156|    2|
|  0|      159|    1|
|  0|      161|    1|
|  0|      180|    1|
|  0|      194|    1|
|  0|      227|    2|
|  1|        5|    1|
|  1|       10|    1|
|  1|       23|    3|
+---+---------+-----+
only showing top 20 rows



In [17]:
main_df.groupBy('t').count().show()

+---+-----+
|  t|count|
+---+-----+
| 31|   26|
| 34|   26|
| 28|   24|
| 26|   28|
| 27|   31|
| 44|   32|
| 12|   31|
| 22|   30|
| 47|   28|
|  1|   33|
| 13|   30|
| 16|   30|
|  6|   28|
|  3|   29|
| 20|   29|
| 40|   28|
| 48|   31|
|  5|   29|
| 19|   32|
| 41|   31|
+---+-----+
only showing top 20 rows



In [22]:
main_df.dropDuplicates().count(), main_df.count()

(1439, 1439)

In [23]:
main_df.na.drop().count()

1439

----------------------
----------------------

----------------------
----------------------

----------------------
----------------------

In [15]:
df0 = main_df.filter("t > 20")
df1 = main_df.filter("t <= 20")

In [16]:
df0 = df0.na.drop()
df0 = df0.orderBy('t', ascending=False)
df0 = df0.dropDuplicates(subset = ['PhoneId'])

In [15]:
df1 = df1.na.drop()
df1 = df1.orderBy('t', ascending=False)
df1 = df1.dropDuplicates(subset = ['PhoneId'])

In [32]:
df0.count(), df1.count()

(22, 22)

In [11]:
def actualisation(base, batch):
    base = base.union(batch.na.drop().orderBy('t', ascending=False).dropDuplicates(subset = ['PhoneId']))
    base = base.orderBy('t', ascending=False).coalesce(1).dropDuplicates(subset = ['PhoneId'])
    return base

In [17]:
df0 = actualisation(df0, df1)

In [18]:
df0.show()

+---+---------+---------+-------+---------+---------+------+
|  t|AntennaId|EventCode|PhoneId|        x|        y|TileId|
+---+---------+---------+-------+---------+---------+------+
| 49|       23|        0|    296| 600054.3| 651917.6|   921|
| 49|      213|        2|    253|599841.56| 650137.1|   433|
| 49|      140|        2|    230| 600220.8| 649846.7|   356|
| 49|      127|        2|    246|600323.94|651923.06|   924|
| 49|      171|        1|    291|601487.94|650147.75|   450|
| 49|      143|        2|    259| 599961.9|651983.94|   920|
| 49|       23|        0|    297| 600054.3| 651917.6|   921|
| 49|       64|        2|    280| 600858.2|649640.56|   308|
| 49|      153|        2|    283|601957.44|650987.94|   670|
| 49|      171|        1|    292|601487.94|650147.75|   450|
| 49|       95|        2|    268| 601408.6|650864.44|   638|
| 49|       65|        2|    299| 600928.2| 649582.7|   282|
| 49|      197|        0|    257| 600768.0|651659.44|   847|
| 49|      217|        0

In [21]:
tile_base = df0.groupBy('TileId').count()
tile_base.show()

+------+-----+
|TileId|count|
+------+-----+
|   921|    2|
|   433|    1|
|   356|    1|
|   924|    2|
|   450|    2|
|   920|    1|
|   308|    1|
|   670|    2|
|   638|    1|
|   282|    1|
|   847|    1|
|    38|    2|
|   732|    1|
|   423|    1|
|   203|    1|
|   463|    1|
|   358|    1|
+------+-----+



Append stream (df1) to base (df0)

In [16]:
df0 = df0.union(df1)

In [34]:
df0.count()

44

On check si on a des `PhoneId` en doublon

In [35]:
df0.groupBy('PhoneId').count().show()

+-------+-----+
|PhoneId|count|
+-------+-----+
|    296|    2|
|    253|    2|
|    230|    2|
|    246|    2|
|    291|    2|
|    259|    2|
|    297|    2|
|    280|    2|
|    283|    2|
|    292|    2|
|    268|    2|
|    299|    2|
|    257|    2|
|    241|    2|
|    273|    2|
|    234|    2|
|    289|    2|
|    245|    2|
|    270|    2|
|    284|    2|
+-------+-----+
only showing top 20 rows



In [65]:
df0.head(3)

[Row(t=46, AntennaId=73, EventCode=0, PhoneId=242, x=602098.6875, y=650285.375, TileId=483),
 Row(t=46, AntennaId=73, EventCode=0, PhoneId=241, x=602098.6875, y=650285.375, TileId=483),
 Row(t=47, AntennaId=73, EventCode=2, PhoneId=242, x=602098.6875, y=650285.375, TileId=483)]

[orderBy](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy) `t`

In [17]:
df0 = df0.orderBy('t', ascending=False)

In [37]:
df0.count()

44

[coalesce](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.coalesce) 1

In [18]:
df0 = df0.coalesce(1)

In [39]:
df0.count()

44

In [ ]:
df0.show()

In [ ]:
df0.dropDuplicates(subset = ['PhoneId']).show(5)

In [19]:
df0 = df0.dropDuplicates(subset = ['PhoneId'])

In [71]:
df0.count()

8

In [20]:
df0.show()

+---+---------+---------+-------+---------+---------+------+
|  t|AntennaId|EventCode|PhoneId|        x|        y|TileId|
+---+---------+---------+-------+---------+---------+------+
| 49|       23|        0|    296| 600054.3| 651917.6|   921|
| 49|      213|        2|    253|599841.56| 650137.1|   433|
| 49|      140|        2|    230| 600220.8| 649846.7|   356|
| 49|      127|        2|    246|600323.94|651923.06|   924|
| 49|      171|        1|    291|601487.94|650147.75|   450|
| 49|      143|        2|    259| 599961.9|651983.94|   920|
| 49|       23|        0|    297| 600054.3| 651917.6|   921|
| 49|       64|        2|    280| 600858.2|649640.56|   308|
| 49|      153|        2|    283|601957.44|650987.94|   670|
| 49|      171|        1|    292|601487.94|650147.75|   450|
| 49|       95|        2|    268| 601408.6|650864.44|   638|
| 49|       65|        2|    299| 600928.2| 649582.7|   282|
| 49|      197|        0|    257| 600768.0|651659.44|   847|
| 49|      217|        0

In [23]:
df0.count()

22

In [25]:
df0.groupBy('TileId').count().show()

Py4JJavaError: An error occurred while calling o2309.showString.
: org.apache.spark.SparkException: Job 20 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:933)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:931)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:931)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2130)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2043)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.spar

In [74]:
main_df.count()

1439

In [21]:
main_df.groupBy('PhoneId').count().count()

22